In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("input/"))

from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB,GaussianNB

from keras.models import Sequential
from keras.layers import Dense, Dropout,Conv2D, Flatten
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,AdaBoostClassifier

import lightgbm
# Any results you write to the current directory are saved as output.

['test.csv', 'sample_submission.csv', 'santander-customer-transaction-prediction.zip', 'train.csv']


/home1/aakash.goel/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
sample_sub = pd.read_csv('input/sample_submission.csv')

In [3]:
sample_sub.head()

,ID_code,target
0,test_0,0
1,test_1,0
2,test_2,0
3,test_3,0
4,test_4,0


In [4]:
print(train.head())
print("==")
print(train.shape)
print("===")
print(test.head())
print("==")
print(test.shape)

   ID_code  target    var_0   var_1    var_2   var_3    var_4   var_5   var_6  \
0  train_0       0   8.9255 -6.7863  11.9081  5.0930  11.4607 -9.2834  5.1187   
1  train_1       0  11.5006 -4.1473  13.8588  5.3890  12.3622  7.0433  5.6208   
2  train_2       0   8.6093 -2.7457  12.0805  7.8928  10.5825 -9.0837  6.9427   
3  train_3       0  11.0604 -2.1518   8.9522  7.1957  12.5846 -1.8361  5.8428   
4  train_4       0   9.8369 -1.4834  12.8746  6.6375  12.2772  2.4486  5.9405   

     var_7   ...     var_190  var_191  var_192  var_193  var_194  var_195  \
0  18.6266   ...      4.4354   3.9642   3.1364   1.6910  18.5227  -2.3978   
1  16.5338   ...      7.6421   7.7214   2.5837  10.9516  15.4305   2.0339   
2  14.6155   ...      2.9057   9.7905   1.6704   1.6858  21.6042   3.1417   
3  14.9250   ...      4.4666   4.7433   0.7178   1.4214  23.0347  -1.2706   
4  19.2514   ...     -1.4905   9.5214  -0.1508   9.1942  13.2876  -1.5121   

   var_196  var_197  var_198  var_199  
0   7.8784

In [5]:
train['target'].value_counts(normalize=True)

0    0.89951
1    0.10049
Name: target, dtype: float64

In [6]:
# pd.isnull(train).sum()

In [7]:
independent_feat = ['var_'+str(i) for i in range(200)]
dependent_feat = 'target'

In [8]:
train[independent_feat].describe()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,10.679914,-1.627622,10.715192,6.796529,11.078333,-5.065317,5.408949,16.545850,0.284162,7.567236,...,3.234440,7.438408,1.927839,3.331774,17.993784,-0.142088,2.303335,8.908158,15.870720,-3.326537
std,3.040051,4.050044,2.640894,2.043319,1.623150,7.863267,0.866607,3.418076,3.332634,1.235070,...,4.559922,3.023272,1.478423,3.992030,3.135162,1.429372,5.454369,0.921625,3.010945,10.438015
min,0.408400,-15.043400,2.117100,-0.040200,5.074800,-32.562600,2.347300,5.349700,-10.505500,3.970500,...,-14.093300,-2.691700,-3.814500,-11.783400,8.694400,-5.261000,-14.209600,5.960600,6.299300,-38.852800
25%,8.453850,-4.740025,8.722475,5.254075,9.883175,-11.200350,4.767700,13.943800,-2.317800,6.618800,...,-0.058825,5.157400,0.889775,0.584600,15.629800,-1.170700,-1.946925,8.252800,13.829700,-11.208475
50%,10.524750,-1.608050,10.580000,6.825000,11.108250,-4.833150,5.385100,16.456800,0.393700,7.629600,...,3.203600,7.347750,1.901300,3.396350,17.957950,-0.172700,2.408900,8.888200,15.934050,-2.819550
75%,12.758200,1.358625,12.516700,8.324100,12.261125,0.924800,6.003000,19.102900,2.937900,8.584425,...,6.406200,9.512525,2.949500,6.205800,20.396525,0.829600,6.556725,9.593300,18.064725,4.836800
max,20.315000,10.376800,19.353000,13.188300,16.671400,17.251600,8.447700,27.691800,10.151300,11.150600,...,18.440900,16.716500,8.402400,18.281800,27.928800,4.272900,18.321500,12.000400,26.079100,28.500700


In [9]:
train[independent_feat].head()

,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [10]:
# sc = StandardScaler()
# train[independent_feat] = sc.fit_transform(train[independent_feat])
# test[independent_feat] = sc.transform(test[independent_feat])

In [11]:
train_1, val = train_test_split(train, test_size=0.20, stratify=train[dependent_feat])
print(train.shape)
print(val.shape)

(200000, 202)
(40000, 202)


In [12]:
train_1[dependent_feat].value_counts(normalize=True)

0    0.899513
1    0.100487
Name: target, dtype: float64

In [13]:
train_data = lightgbm.Dataset(train_1[independent_feat], label=train_1[dependent_feat])
val_data = lightgbm.Dataset(val[independent_feat], label=val[dependent_feat])

In [14]:
val_data

In [15]:
## v-2 param
random_state = 42
param = {
    "objective" : "binary",
    "metric" : "auc",
    "boosting": 'gbdt',
    "max_depth" : 90,
    "num_leaves" : 15,
    "learning_rate" : 0.01000123,
    "bagging_freq": 5,
    "bagging_fraction" : 0.4,
    "feature_fraction" : 0.05,
    "min_data_in_leaf": 150,
    "min_sum_heassian_in_leaf": 15,
    "tree_learner": "voting",
    "boost_from_average": "false",
    "lambda_l1" : 10,
    "lambda_l2" : 10,
    "bagging_seed" : random_state,
    "verbosity" : 1,
    "seed": random_state
}

# param = {
#     'bagging_freq': 5,
#     'bagging_fraction': 0.335,
#     'boost_from_average':'false',
#     'boost': 'gbdt',
#     'feature_fraction': 0.041,
#     'learning_rate': 0.083,
#     'max_depth': -1,
#     'metric':'auc',
#     'min_data_in_leaf': 80,
#     'min_sum_hessian_in_leaf': 10.0,
#     'num_leaves': 13,
#     'num_threads': 8,
#     'tree_learner': 'serial',
#     'objective': 'binary', 
#     'verbosity': -1
# }

# param = {
#     'bagging_freq': 5,
#     'bagging_fraction': 0.335,
#     'boost_from_average':'false',
#     'boost': 'gbdt',
#     'feature_fraction': 0.041,
#     'learning_rate': 0.083,
#     'max_depth': -1,
#     'metric':'auc',
#     'min_data_in_leaf': 50,
#     'min_sum_hessian_in_leaf': 10.0,
#     'num_leaves': 11,
#     'num_threads': 8,
#     'tree_learner': 'serial',
#     'objective': 'binary', 
#     'verbosity': -1
# }

# param = {
#     'bagging_freq': 5,
#     'bagging_fraction': 0.335,
#     'boost_from_average':'false',
#     'boost': 'gbdt',
#     'feature_fraction': 0.041,
#     'learning_rate': 0.0083,
#     'max_depth': -1,
#     'metric':'auc',
#     'min_data_in_leaf': 80,
#     'min_sum_hessian_in_leaf': 10.0,
#     'num_leaves': 13,
#     'num_threads': 30,
#     'tree_learner': 'serial',
#     'objective': 'binary', 
#     'verbosity': -1
# }

In [16]:
model = lightgbm.train(param,
                       train_data,
                       valid_sets=val_data,
                       num_boost_round=100000,
                       verbose_eval=5000, 
                       early_stopping_rounds = 10000
                       )

Training until validation scores don't improve for 10000 rounds.
[5000]	valid_0's auc: 0.898875
[10000]	valid_0's auc: 0.899644
[15000]	valid_0's auc: 0.898994
Early stopping, best iteration is:
[7057]	valid_0's auc: 0.899748


In [18]:
train_pred = model.predict(train_1[independent_feat])
val_pred = model.predict(val[independent_feat])
test_pred = model.predict(test[independent_feat])

In [19]:
print(train_pred)
print("=======")
print(val_pred)

[0.25615863 0.00172274 0.03219209 ... 0.04084708 0.1282732  0.06411532]
[0.02771139 0.0098377  0.0032153  ... 0.17341252 0.00537678 0.04516144]


In [20]:
from sklearn.metrics import roc_auc_score
print("Train auc {}".format(roc_auc_score(train_1[dependent_feat],train_pred)))
print("Val auc {}".format(roc_auc_score(val[dependent_feat],val_pred)))

Train auc 0.9322243276759087
Val auc 0.8997415783782587


In [28]:
result = pd.DataFrame({'ID_code':test['ID_code'],'target':list(test_pred)})
result.head()

,ID_code,target
0,test_0,0.088182
1,test_1,0.270862
2,test_2,0.169902
3,test_3,0.139457
4,test_4,0.045031


In [29]:
result.to_csv('output/lgbm_woscale_woaugment.csv',index=False)

References

1) https://www.kaggle.com/ezietsman/simple-python-lightgbm-example